<a href="https://colab.research.google.com/github/prateekchandrajha/mastering-ml-algorithms/blob/main/Deploy_ML_Models_as_Service_with_FastAPI_%26_ColabCode_from_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colabcode
!pip install fastapi

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 61kB 3.1MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.1-cp36-none-any.whl size=18822 sha256=3ce53532eb2c65342d9067a379c7b3f085e0ef22011cfe890e5417befeb08429
  Stored in directory: /root/.cache/pip/wheels/94/01/05/d39efb8f6b40a411354b4168ca9dda99e6f8d586e458e97551
Successfully built pyngrok
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
  Found existing installation: nest-asyncio 1.5.1
    Uninstalling nest-asyncio-1.5.1:
      Successfully uninstalled nest-asyncio-1.5.1
     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 9.2MB 12.1MB/s 


In [2]:
from colabcode import ColabCode
from fastapi import FastAPI

In [3]:
cc = ColabCode(port=12000, code=False)

In [4]:
app = FastAPI()

@app.get("/") # no end point, just the root, a very light-weight app
async def read_root():
  return {"message": "Subscribe to @1littlecoder"}

In [5]:
cc.run_app(app=app)

INFO:     Started server process [62]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "http://8605de063257.ngrok.io" -> "http://localhost:12000"
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET / HTTP/1.1" 200 OK
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [62]


In [6]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pickle

iris = load_iris()
model = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
model_f = model.fit(X_train, y_train)

print("Model score: ", model.score(X_train, y_train))
print("Test Accuracy: ", model.score(X_test, y_test))

pickle.dump(model_f, open("model_gb.pkl", "wb"))

Model score:  0.9555555555555556
Test Accuracy:  1.0


In [7]:
%%writefile models.py
from pydantic import BaseModel, conlist
from typing import List


class Iris(BaseModel):
    data: List[conlist(float, min_items=4, max_items=4)]

Writing models.py


In [8]:
import pickle
import logging
from fastapi import FastAPI
from models import Iris

app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# # Initialize logging
# my_logger = logging.getLogger()
# my_logger.setLevel(logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG, filename='logs.log')

model = None

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_gb.pkl", "rb"))

@app.post("/api", tags=["prediction"])
async def get_predictions(iris: Iris):
    try:
        data = dict(iris)['data']
        print(data)
        iris_types = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }
        prediction = list(map(lambda x: iris_types[x], model.predict(data).tolist()))
        log_proba = model.predict_log_proba(data).tolist()
        return {"prediction": prediction, "log_proba": log_proba}
    except:
        my_logger.error("Something went wrong!")
        return {"prediction": "error"}

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "http://b6c41d4878a9.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [62]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "POST /api HTTP/1.1" 422 Unprocessable Entity
[[4.5, 3.1, 1.0, 5.0]]
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "POST /api HTTP/1.1" 200 OK
[[0.5, 3.1, 1.0, 5.0]]
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "POST /api HTTP/1.1" 200 OK
[[0.5, 3.1, 199.0, 5.0]]
INFO:     2402:e280:2200:5992:45e3:513c:4120:f28:0 - "POST /api HTTP/1.1" 200 OK
